In [ ]:
using PandaRobot
using GuSTO

In [ ]:
robot = PandaBot()
model = PandaKin()
env = Factory()

th_init = [0;0;0; -0.1; 0; 0.5; 0]
th_goal = [0.5;0.5;0.5; -1.; 0; 0.5; 0]

x_init, x_goal = zeros(model.x_dim), zeros(model.x_dim)
for i in 1:model.num_joints
    x_init[2*i-1:2*i] = [cos(th_init[i]); sin(th_init[i])]
    x_goal[2*i-1:2*i] = [cos(th_goal[i]); sin(th_goal[i])]
end

N = 80
tf_guess = 50.

In [ ]:
PD = ProblemDefinition(robot, model, env, x_init, x_goal);
TOPgusto = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOPgusto)
solve_SCP!(TOSgusto, TOPgusto, solve_gusto_cvx!, init_traj_so1, "Gurobi", OutputFlag=0)

In [ ]:
X = TOSgusto.SCPS.traj.X
U = TOSgusto.SCPS.traj.U
SCPP = SCPProblem(TOPgusto)
joint_traj = zeros(model.num_joints,N)

for k in 1:N
   joint_traj[:,k] = get_configuration(X[:,k],SCPP) 
end
for idx in 1:model.num_joints
    PyPlot.plot(joint_traj[idx,:])
end

In [ ]:
using MeshCat
using MeshCatMechanisms

using RigidBodySim
using RigidBodyDynamics

vis = Visualizer()

pd = robot.pan
mvis = MechanismVisualizer(
    pd.mechanism,
    URDFVisuals(PandaRobot.urdfpath(), package_path=[dirname(dirname(PandaRobot.urdfpath()))]),
    vis);

q = [ 
    [0;0;0;0;0;0;0;0;0],
    [1;1;1;1;1;1;1;0.05;0.05],
    [0;0.5;0.7;0.5;0.3;0.8;0.2;0.02;0.02]
    ];

setanimation!(mvis, 1:length(q), q)

plot_in_cell = false
plot_in_cell ? IJuliaCell(vis) : open(vis)
#sleep(1)